# Feature overload
this is just to see what happens if a ton of features are shoved into the model. This will include avg embedding per news article (300 dimensions), keywords tfidf (50 dimensions), and article text tfidf (2000 dimensions). Didn't work. Probably overfit.

In [76]:
from __future__ import print_function
from pprint import pprint
from time import time
import logging
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
%matplotlib inline
nlp = spacy.load('en')
from gensim.models import KeyedVectors

# Load Data

In [5]:
project_data = pd.read_pickle('../Data/EWS_Published Project_Listing_DD.pkl')

In [6]:
data = pd.read_pickle('../Data/Feedly_Processed_DF_cleaned.pkl')

In [7]:
train_data = pd.read_csv('../Data/Labeled_Data/sectors.csv')

In [58]:
data_with_sectors = data.merge(train_data[['article_id', 'Sectors', 'cl_Sector', 'top_sector']],
                    on='article_id')

# fix sector labels

In [59]:
data_with_sectors.astype({'Sectors':str})
data_with_sectors[data_with_sectors['Sectors'] == 'transport'] = 'Transport'
data_with_sectors[data_with_sectors['Sectors'] == 'Not a project'] = None
data_with_sectors[data_with_sectors['Sectors'] == '0'] = None
data_with_sectors['Sectors'] = data_with_sectors['Sectors'].str.upper()
data_with_sectors['Sectors'] = data_with_sectors['Sectors'].apply(lambda x: None if not x else x.replace('TRADE AND INDUSTRY','INDUSTRY AND TRADE'))
data_with_sectors['Sectors'] = data_with_sectors['Sectors'].apply(lambda x: None if not x else x.replace('\n',''))
data_with_sectors['Sectors'] = data_with_sectors['Sectors'].apply(lambda x: None if not x else ','.join([x.strip() for x in x.split(',')]))

In [ ]:
#merge labels back with dataframe

In [145]:
classes = data_with_sectors['Sectors'].str.get_dummies(',').columns

In [65]:
data_with_sectors = pd.concat([data_with_sectors, data_with_sectors['Sectors'].str.get_dummies(',')], axis=1)

In [73]:
#Fix article_keywords
data_with_sectors['article_keywords'] = data_with_sectors['article_keywords'].apply(lambda x: [] if not x or type(x) != list else x)

# External Language dependencies

In [78]:
w2v = KeyedVectors.load_word2vec_format(r'../dependencies/wiki-news-300d-1M.vec')
with open(r'stop_words.txt', 'r') as f:
    stop_words = set(f.read().strip().split("\n"))

# Features

In [81]:
data_with_sectors['spacy_text'] = data_with_sectors['article_text'].apply(lambda x: nlp('') if not x else nlp(x))

In [124]:
def create_embedding(spac_text):
    embed=[]
    for tok in spac_text:
        if tok not in stop_words and tok.text.lower() in w2v:
            embed.append(w2v[tok.text.lower()])
    return np.mean(embed, axis=0) if len(embed) > 0 else [0]*300

In [132]:
data_with_sectors['avg_embedding'] = data_with_sectors['spacy_text'].apply(lambda x: create_embedding(x))

In [86]:
data_with_sectors['text_without_sw'] = data_with_sectors['spacy_text'].apply(lambda x: ' '.join([y.text for y in x if y.text.lower() not in stop_words]))

In [88]:
data_with_sectors['keywords'] = data_with_sectors['article_keywords'].apply(lambda x: ' '.join(x))

# Features (embedding, text, keywords)

In [149]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_with_sectors, test_size=.3, random_state=42)

In [175]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv_1 = TfidfVectorizer(ngram_range = (1,3), max_features=2000)
tv_2 = TfidfVectorizer(ngram_range = (1,2), max_features=50)
txt_feat = tv_1.fit_transform(train['text_without_sw'])
kw_feat = tv_2.fit_transform(train['keywords'])

In [184]:
train_x = np.hstack([ np.vstack(train['avg_embedding'])])
train_y = train[classes].values

In [206]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [207]:
test_txt_feat = tv_1.transform(test['text_without_sw'])
test_kw_feat = tv_2.transform(test['keywords'])

In [208]:
test_x =  np.hstack([np.vstack(test['avg_embedding'])])
test_y = test[classes].values

In [226]:
pred = rf.predict_proba(test_x)

In [227]:
pred = np.transpose([[1 if y[1] > 0.5 else 0 for y in x] for x in pred])

In [228]:
len(pred)

89

In [211]:
from sklearn import metrics

In [212]:
# print (metrics.classification_report(test_y, pred))

In [231]:

metrics.accuracy_score(test_y, pred)

0.39325842696629215